In [1]:
from app.core.config import settings
from app.database.mysql import get_mysql_connection
mysql_connection = get_mysql_connection()

2024-12-21 13:28:24,338 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_URI
2024-12-21 13:28:24,545 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_USER
2024-12-21 13:28:24,721 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_PASSWORD
2024-12-21 13:28:24,917 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_DB_NAME
2024-12-21 13:28:25,086 - fastapi_project - INFO - Retrieved parameter: /tailorlink/openai/OPENAI_API_KEY
2024-12-21 13:28:25,261 - fastapi_project - INFO - Retrieved parameter: /tailorlink/milvus/MILVUS_URI
2024-12-21 13:28:25,427 - fastapi_project - INFO - Retrieved parameter: /tailorlink/milvus/MILVUS_TOKEN
2024-12-21 13:28:25,620 - fastapi_project - INFO - Retrieved parameter: /tailorlink/langchain/LANGCHAIN_API_KEY
2024-12-21 13:28:25,785 - fastapi_project - INFO - Retrieved parameter: /tailorlink/langchain/LANGCHAIN_ENDPOINT
2024-12-21 13:28:26,004 - fastapi_project - INFO - Co

In [2]:
from operator import itemgetter
from typing import List

from langchain_openai.chat_models import ChatOpenAI

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]


history = get_by_session_id("1")
history.add_message(AIMessage(content="hello"))
print(store)  # noqa: T201

{'1': InMemoryHistory(messages=[AIMessage(content='hello', additional_kwargs={}, response_metadata={})])}


In [3]:
from typing import Optional

from langchain_community.chat_models import ChatAnthropic
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory


prompt = ChatPromptTemplate.from_messages([
    ("system", "You're my best friend. You must play with me"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | ChatOpenAI(model="gpt-4o-mini")

chain_with_history = RunnableWithMessageHistory(
    chain,
    # Uses the get_by_session_id function defined in the example
    # above.
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

print(chain_with_history.invoke(  # noqa: T201
    {"question": "내 이름은 마바사야?"},
    config={"configurable": {"session_id": "foo"}}
))

# Uses the store defined in the example above.
print(store)  # noqa: T201

print(chain_with_history.invoke(  # noqa: T201
    {"ability": "math", "question": "내 이름은 뭐야?"},
    config={"configurable": {"session_id": "foo"}}
))

print(store)  # noqa: T201

C:\Users\ChoiYounKyu\AppData\Local\Temp\ipykernel_18496\276362656.py:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chain = prompt | ChatOpenAI(model="gpt-4o-mini")


content='안녕하세요, 마바사! 만나서 반가워요. 어떻게 도와줄까요?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 28, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None} id='run-1b7781bd-1e61-4f64-a245-9530854b1a94-0'
{'1': InMemoryHistory(messages=[AIMessage(content='hello', additional_kwargs={}, response_metadata={})]), 'foo': InMemoryHistory(messages=[HumanMessage(content='내 이름은 마바사야?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 마바사! 만나서 반가워요. 어떻게 도와줄까요?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 28, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tok

In [4]:
res =chain_with_history.invoke(  # noqa: T201
    {"ability": "math", "question": "내 이름은 뭐야?"},
    config={"configurable": {"session_id": "foo1"}}
)

In [7]:
res.content

'안녕! 너의 이름은 내가 알 수 없지만, 너가 원하는 이름을 알려주면 그 이름으로 불러줄게! 너의 이름이 뭐야?'

In [13]:
from app.database.chat_history import ChatHistoryManager, Session
from langchain_core.messages import HumanMessage
# 세션 및 매니저 초기화
session = Session()
chat_history_manager = ChatHistoryManager(session)

# 대화 기록 예제
chat_history_manager.save_message(
    session_id="user1a",
    message=HumanMessage(content="안녕하세요"),
    message_type="human"
)
print(chat_history_manager.load_history("user1"))


[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={})]


In [14]:
print(chat_history_manager.load_history("user1a"))

[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={})]
